<a href="https://colab.research.google.com/gist/sarazerie/29650ec00e46ea54cfa25a62e945a6c2/generalcvextract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing** **Extraction** **Dependencies** **Libraries**

In [4]:
!pip install pdfminer.six rapidfuzz phonenumbers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 80.7 MB/s eta 0:00:00


**General CV Extraction Process**

In [11]:
import os
import re
import csv
import json
from pdfminer.high_level import extract_text
from pdfminer.pdfparser import PDFSyntaxError
from rapidfuzz import fuzz
import phonenumbers


# REGEX DEFINITIONS


EMAIL_RE = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
URL_RE = re.compile(r"(?i)\b((?:https?://|www\.)[^\s<>()]+)")
PHONE_RE = re.compile(r"\+?\d[\d\s().-]{7,}\d")
DEGREE_RE = re.compile(r"(PhD|Doctor|Master.?s|Bachelor.?s|BSc|MSc|BS|MS|BA|MA|Associate)", re.IGNORECASE)
GPA_RE = re.compile(r"GPA[:\s]*([0-4]\.\d{1,2})", re.IGNORECASE)

SECTION_RE = re.compile(r"^[A-Z][A-Z &/]{3,}$")  # Detect headings (e.g SKILLS)

UNIVERSITY_RE = re.compile(
    r"([A-Z][A-Za-z0-9&.,'() -]{1,100}?(University|College|Institute|School|Academy))",
    re.IGNORECASE
)

SKILLS_DB = [
    "Python","Java","C++","C","SQL","Machine Learning","Deep Learning","NLP",
    "Data Analysis","Data Science","Computer Vision",
    "Network Security","Cryptography","Post-Quantum Cryptography",
    "Software Engineering","Project Management","Teaching","Research",
    "Algorithms","Docker","Linux","TensorFlow","PyTorch"
]


# SAFE PDF TEXT EXTRACTION

def extract_text_pdf(path):
    try:
        text = extract_text(path)
    except PDFSyntaxError:
        text = ""
    except Exception as e:
        print(f"PDF error in {path}: {e}")
        text = ""

    text = re.sub(r"\(cid:\d+\)", "", text)
    return text


# NAME EXTRACTION

def extract_name(text):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    for line in lines[:5]:
        if EMAIL_RE.search(line) or PHONE_RE.search(line) or re.search(r"\d", line):
            continue
        if "cv" in line.lower() or "resume" in line.lower():
            continue
        if 2 <= len(line.split()) <= 4:  # Usually a name length
            return line
    return ""


# EMAIL / PHONE / WEBSITE

def extract_emails(text):
    return list(set(EMAIL_RE.findall(text)))

def extract_websites(text):
    return list(set([m.group(1) for m in URL_RE.finditer(text)]))

def extract_phones(text):
    numbers = []
    for match in phonenumbers.PhoneNumberMatcher(text, None):
        formatted = phonenumbers.format_number(
            match.number, phonenumbers.PhoneNumberFormat.E164
        )
        numbers.append(formatted)
    return list(dict.fromkeys(numbers))


# EDUCATION SECTION

def extract_education(text):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    edu_lines = []
    in_section = False

    # Locate EDUCATION section
    for line in lines:
        if "EDUCATION" in line.upper():
            in_section = True
            continue
        if in_section:
            if SECTION_RE.match(line):  # reached next major section
                break
            edu_lines.append(line)

    # Merge and split entries
    block = " ".join(edu_lines)
    raw_entries = re.split(r"•|▪|●|\n| {2,}", block)

    entries = []

    for entry in raw_entries:
        entry = entry.strip()
        if len(entry) < 3:
            continue

        degree = None
        university = None
        gpa = None

        deg_match = re.search(DEGREE_RE, entry)
        if deg_match:
            degree = deg_match.group(0)

        uni_match = re.search(UNIVERSITY_RE, entry)
        if uni_match:
            university = uni_match.group(0)

        gpa_match = re.search(GPA_RE, entry)
        if gpa_match:
            gpa = gpa_match.group(1)

        if degree or university:
            entries.append({
                "degree": degree,
                "university": university,
                "gpa": gpa
            })

    return entries


# SKILLS EXTRACTION

def extract_skills(text):
    found = set()
    text_lower = text.lower()

    # Vocabulary fuzzy match
    for skill in SKILLS_DB:
        if fuzz.partial_ratio(skill.lower(), text_lower) >= 70:
            found.add(skill)

    # Skills section auto-detection
    lines = text.splitlines()
    skill_block = ""
    for i, line in enumerate(lines):
        if "skill" in line.lower():
            skill_block = " ".join(lines[i:i+5])
            break

    raw = re.split(r"[•,;|\n]", skill_block)
    for s in raw:
        s = s.strip()
        if 2 < len(s) < 40:
            found.add(s)

    return sorted(found)


# PARSE SINGLE PDF

def parse_pdf(path):
    text = extract_text_pdf(path)

    return {
        "file": os.path.basename(path),
        "name": extract_name(text),
        "emails": extract_emails(text),
        "phones": extract_phones(text),
        "websites": extract_websites(text),
        "education": extract_education(text),
        "skills": extract_skills(text)
    }


# PROCESS FOLDER

def process_folder(pdf_folder, output_csv, output_json):
    all_profiles = []

    for filename in os.listdir(pdf_folder):
        if filename.lower().endswith(".pdf"):
            path = os.path.join(pdf_folder, filename)
            profile = parse_pdf(path)
            all_profiles.append(profile)

            print("\n===========================")
            print(f"📄 FILE: {filename}")
            print(f"Name: {profile['name']}")
            print("Emails:", profile['emails'])
            print("Phones:", profile['phones'])
            print("Websites:", profile['websites'])
            print("Education:")
            for e in profile['education']:
                print(f"  - {e['degree']} | {e['university']} | GPA: {e['gpa']}")
            print("Skills:", ", ".join(profile['skills']))

    # Save CSV
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["File", "Name", "Emails", "Phones", "Websites", "Education", "Skills"])

        for p in all_profiles:
            edu_str = "; ".join([
                f"{e['degree']} | {e['university']} | GPA: {e['gpa']}"
                for e in p["education"]
            ])

            writer.writerow([
                p["file"],
                p["name"],
                ", ".join(p["emails"]),
                ", ".join(p["phones"]),
                ", ".join(p["websites"]),
                edu_str,
                ", ".join(p["skills"])
            ])

    # Save JSON
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_profiles, f, indent=2, ensure_ascii=False)

    print("\nSaved CSV and JSON successfully!")


print(process_folder('/content/drive/MyDrive/cvs/anas','/content/drive/MyDrive/cvs/profiles.csv','/content/drive/MyDrive/cvs/profiles.json' ))
print(process_folder('/content/drive/MyDrive/cvs/academic_cvs','/content/drive/MyDrive/cvs/academic_profiles.csv','/content/drive/MyDrive/cvs/academic_profiles.json'))



📄 FILE: Anas_CV.pdf
Name: ANAS MELHEM
Emails: ['a.melhem@ptuk.edu.ps']
Phones: ['+970599320207']
Websites: ['https://anas-melhem.github.io']
Education:
  - PhD | PhD in Computer Engineering Eastern Mediterranean University | GPA: None
  - Master’s | s in Electronics and Computer Engineering Al-Quds University | GPA: None
  - Bachelor’s | s in Electrical Engineering Palestine Technical University | GPA: None
Skills: Algorithms, C, Computer Vision, Cryptanalysis, Cryptographic Algorithm Design, Cryptography, Data Analysis, Data Science, Java, Network Security, Post-Quantum Cryptography, Research, SKILLS, Software Engineering, Teaching

Saved CSV and JSON successfully!
None

📄 FILE: Sara_Saifi_1.pdf
Name: SARA SAIFI
Emails: ['sara.saifi@student.ptuk.edu.ps']
Phones: ['+970599843638']
Websites: ['https://sara-saifi.github.io']
Education:
  - Bachelor’s | s in Software Engineering University | GPA: 2.55
Skills: C, C++, Deep Learning, Linux, Machine Learning, Python, Research, SKILLS   Linu

**Same above code but more accurate in Education section**

In [15]:
import os
import re
import csv
import json
from pdfminer.high_level import extract_text
from pdfminer.pdfparser import PDFSyntaxError
from rapidfuzz import fuzz
import phonenumbers


# REGEX DEFINITIONS

EMAIL_RE = re.compile(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
URL_RE = re.compile(r"(?i)\b((?:https?://|www\.)[^\s<>()]+)")
PHONE_RE = re.compile(r"\+?\d[\d\s().-]{7,}\d")
SECTION_RE = re.compile(r"^[A-Z][A-Z &/]{3,}$")  # Detect headings (SKILLS, etc.)

# Degree regex
DEGREE_RE = re.compile(r"(PhD|Doctor|Master.?s|Bachelor.?s|BSc|MSc|BS|MS|BA|MA|Associate)", re.IGNORECASE)

# GPA
GPA_RE = re.compile(r"GPA[:\s]*([0-4]\.\d+)", re.IGNORECASE)

# University robust regex
UNIVERSITY_RE = re.compile(
    r"((?:University of [A-Z][A-Za-z ]+)|"
    r"(?:[A-Z][A-Za-z ]+ University)|"
    r"(?:[A-Z][A-Za-z ]+ College)|"
    r"(?:[A-Z][A-Za-z ]+ Institute)|"
    r"(?:[A-Z][A-Za-z ]+ Academy))",
    re.IGNORECASE
)

# Skills database (expandable)
SKILLS_DB = [
    "Python","Java","C++","C","SQL","Machine Learning","Deep Learning","NLP",
    "Data Analysis","Data Science","Computer Vision",
    "Network Security","Cryptography","Post-Quantum Cryptography",
    "Software Engineering","Project Management","Teaching","Research",
    "Algorithms","Docker","Linux","TensorFlow","PyTorch"
]


# PDF TEXT EXTRACTION

def extract_text_pdf(path):
    try:
        text = extract_text(path)
    except PDFSyntaxError:
        text = ""
    except Exception as e:
        print(f"PDF error in {path}: {e}")
        text = ""
    text = re.sub(r"\(cid:\d+\)", "", text)
    return text


# NAME EXTRACTION

def extract_name(text):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    for line in lines[:8]:
        if EMAIL_RE.search(line) or PHONE_RE.search(line) or re.search(r"\d", line):
            continue
        if "cv" in line.lower() or "resume" in line.lower():
            continue
        if 2 <= len(line.split()) <= 4:
            return line
    return ""


# EMAIL / PHONE / WEBSITE

def extract_emails(text):
    return list(set(EMAIL_RE.findall(text)))

def extract_websites(text):
    return list(set([m.group(1) for m in URL_RE.finditer(text)]))

def extract_phones(text):
    numbers = []
    for match in phonenumbers.PhoneNumberMatcher(text, None):
        formatted = phonenumbers.format_number(
            match.number, phonenumbers.PhoneNumberFormat.E164
        )
        numbers.append(formatted)
    return list(dict.fromkeys(numbers))


# EDUCATION EXTRACTION

def extract_education(text):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    edu_lines = []
    in_section = False

    for line in lines:
        if "EDUCATION" in line.upper():
            in_section = True
            continue
        if in_section:
            if SECTION_RE.match(line):
                break
            edu_lines.append(line)

    block = " ".join(edu_lines)
    raw_entries = re.split(r"•|▪|●|\n| {2,}", block)
    entries = []

    for entry in raw_entries:
        entry = entry.strip()
        if len(entry) < 3:
            continue

        degree = None
        university = None
        gpa = None

        # Degree
        deg_label = re.search(r"Degree[:\s]+(.+)", entry, re.IGNORECASE)
        if deg_label:
            degree = deg_label.group(1).strip()
        elif re.search(DEGREE_RE, entry):
            degree = re.search(DEGREE_RE, entry).group(0)

        # University
        uni_label = re.search(r"University[:\s]+(.+)", entry, re.IGNORECASE)
        if uni_label:
            university = uni_label.group(1).strip()
        else:
            uni_match = UNIVERSITY_RE.search(entry)
            if uni_match:
                university = uni_match.group(1).strip()

        # Avoid wrong matches like "Software Engineering University"
        if university and len(university.split()) <= 2:
            if "Engineering" in university or "Science" in university:
                university = None

        # GPA
        gpa_match = re.search(GPA_RE, entry)
        if gpa_match:
            gpa = gpa_match.group(1)

        if degree or university:
            entries.append({
                "degree": degree,
                "university": university,
                "gpa": gpa
            })
    return entries


# SKILLS EXTRACTION

def extract_skills(text):
    found = set()
    text_lower = text.lower()

    # Vocabulary fuzzy match
    for skill in SKILLS_DB:
        if fuzz.partial_ratio(skill.lower(), text_lower) >= 70:
            found.add(skill)

    # Detect skill section
    lines = text.splitlines()
    skill_block = ""
    for i, line in enumerate(lines):
        if "skill" in line.lower():
            skill_block = " ".join(lines[i:i+5])
            break
    raw = re.split(r"[•,;|\n]", skill_block)
    for s in raw:
        s = s.strip()
        if 2 < len(s) < 40:
            found.add(s)

    return sorted(found)


# PARSE SINGLE PDF

def parse_pdf(path):
    text = extract_text_pdf(path)
    return {
        "file": os.path.basename(path),
        "name": extract_name(text),
        "emails": extract_emails(text),
        "phones": extract_phones(text),
        "websites": extract_websites(text),
        "education": extract_education(text),
        "skills": extract_skills(text)
    }


# PROCESS FOLDER OF CVS

def process_folder(pdf_folder, output_csv, output_json):
    all_profiles = []

    for filename in os.listdir(pdf_folder):
        if filename.lower().endswith(".pdf"):
            path = os.path.join(pdf_folder, filename)
            profile = parse_pdf(path)
            all_profiles.append(profile)

            print("\n===========================")
            print(f"📄 FILE: {filename}")
            print(f"Name: {profile['name']}")
            print("Emails:", profile['emails'])
            print("Phones:", profile['phones'])
            print("Websites:", profile['websites'])
            print("Education:")
            for e in profile['education']:
                print(f"  - {e['degree']} | {e['university']} | GPA: {e['gpa']}")
            print("Skills:", ", ".join(profile['skills']))

    # Save CSV
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["File", "Name", "Emails", "Phones", "Websites", "Education", "Skills"])
        for p in all_profiles:
            edu_str = "; ".join([f"{e['degree']} | {e['university']} | GPA: {e['gpa']}" for e in p["education"]])
            writer.writerow([
                p["file"], p["name"], ", ".join(p["emails"]), ", ".join(p["phones"]),
                ", ".join(p["websites"]), edu_str, ", ".join(p["skills"])
            ])

    # Save JSON
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_profiles, f, indent=2, ensure_ascii=False)

    print("\nSaved CSV and JSON successfully!")


print(process_folder('/content/drive/MyDrive/cvs/anas','/content/drive/MyDrive/cvs/profiles.csv','/content/drive/MyDrive/cvs/profiles.json' ))
print(process_folder('/content/drive/MyDrive/cvs/academic_cvs','/content/drive/MyDrive/cvs/academic_profiles.csv','/content/drive/MyDrive/cvs/academic_profiles.json'))



📄 FILE: Anas_CV.pdf
Name: ANAS MELHEM
Emails: ['a.melhem@ptuk.edu.ps']
Phones: ['+970599320207']
Websites: ['https://anas-melhem.github.io']
Education:
  - PhD | PhD in Computer Engineering Eastern Mediterranean University | GPA: None
  - Master’s | Quds University | GPA: None
  - Bachelor’s | s in Electrical Engineering Palestine Technical University | GPA: None
Skills: Algorithms, C, Computer Vision, Cryptanalysis, Cryptographic Algorithm Design, Cryptography, Data Analysis, Data Science, Java, Network Security, Post-Quantum Cryptography, Research, SKILLS, Software Engineering, Teaching

Saved CSV and JSON successfully!
None

📄 FILE: Sara_Saifi_1.pdf
Name: SARA SAIFI
Emails: ['sara.saifi@student.ptuk.edu.ps']
Phones: ['+970599843638']
Websites: ['https://sara-saifi.github.io']
Education:
  - Bachelor’s in Software Engineering University: An-Najah National University Start Date: 09/2021 End Date: 06/2025 GPA: 2.55 | An-Najah National University Start Date: 09/2021 End Date: 06/2025 G